# 🧠 V2 Location-Based Model Training

Train a model that uses **latitude/longitude** as primary features instead of country.

**Key Changes from V1:**
- Uses exact coordinates, not country_encoded
- Includes climate_zone and hemisphere features
- Works for any location on Earth

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import joblib
import json
import plotly.express as px
import plotly.graph_objects as go

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"✅ Using device: {device}")

✅ Using device: cuda


## 1. Load and Prepare Data

In [2]:
# Load processed weather data from V1
df = pd.read_csv('../../data/processed/weather_cleaned.csv', parse_dates=['date'])
print(f"📊 Loaded {len(df):,} rows")

# Load V2 location stats (with climate zones)
location_stats = pd.read_csv('../models/location_stats.csv')
print(f"🌍 Location stats: {len(location_stats)} countries")

📊 Loaded 102,652 rows
🌍 Location stats: 186 countries


In [3]:
# Merge climate zone info into main dataset
df = df.merge(
    location_stats[['country', 'hemisphere_encoded', 'climate_zone_encoded', 'abs_latitude', 'latitude_normalized']],
    on='country',
    how='left'
)

# Drop any rows with missing values
df = df.dropna()
print(f"📊 After merge: {len(df):,} rows")
df.head()

📊 After merge: 102,652 rows


,country,date,temperature_celsius,humidity,pressure_mb,wind_kph,precip_mm,cloud,uv_index,latitude,...,temp_lag_14,temp_lag_30,temp_rolling_mean_7,temp_rolling_mean_14,temp_rolling_std_7,country_encoded,hemisphere_encoded,climate_zone_encoded,abs_latitude,latitude_normalized
0,Afghanistan,2024-06-15,22.4,38.0,1009.0,9.4,0.0,27.0,6.0,34.52,...,22.5,24.3,25.128571,23.550000,2.244782,0,1,1,34.517341,0.383526
1,Afghanistan,2024-06-16,26.3,27.0,1006.0,17.6,0.0,31.0,7.0,34.52,...,26.5,15.0,24.885714,23.542857,2.456575,0,1,1,34.517341,0.383526
2,Afghanistan,2024-06-17,27.0,27.0,1006.0,11.5,0.0,16.0,7.0,34.52,...,26.1,19.5,25.157143,23.528571,2.498571,0,1,1,34.517341,0.383526
3,Afghanistan,2024-06-18,26.8,19.0,1002.0,21.6,0.0,3.0,7.0,34.52,...,24.3,16.9,25.400000,23.592857,2.595509,0,1,1,34.517341,0.383526
4,Afghanistan,2024-06-19,26.3,18.0,1001.0,31.0,0.0,0.0,7.0,34.52,...,19.0,14.1,25.400000,23.771429,2.595509,0,1,1,34.517341,0.383526


## 2. Define V2 Features

In [4]:
# V2 feature columns - location-centric (no country_encoded!)
FEATURE_COLS = [
    # Geographic
    'latitude', 'longitude', 'abs_latitude', 'latitude_normalized',
    'hemisphere_encoded', 'climate_zone_encoded',
    
    # Temporal
    'month', 'day_of_month', 'day_of_week', 'day_of_year', 'quarter', 'is_weekend',
    
    # Cyclical
    'month_sin', 'month_cos', 'day_sin', 'day_cos', 'day_of_year_sin', 'day_of_year_cos',
    
    # Lag features
    'temp_lag_1', 'temp_lag_2', 'temp_lag_3', 'temp_lag_7', 'temp_lag_14', 'temp_lag_30',
    
    # Rolling stats
    'temp_rolling_mean_7', 'temp_rolling_mean_14', 'temp_rolling_std_7'
]

TARGET = 'temperature_celsius'

print(f"📊 Total features: {len(FEATURE_COLS)}")
print(f"🎯 Target: {TARGET}")

📊 Total features: 27
🎯 Target: temperature_celsius


In [5]:
# Verify all features exist
missing = [c for c in FEATURE_COLS if c not in df.columns]
if missing:
    print(f"❌ Missing columns: {missing}")
else:
    print("✅ All feature columns present")

✅ All feature columns present


## 3. Train/Test Split

In [6]:
# Sort by date for time-based split
df = df.sort_values('date')

# 80/20 time-based split
split_idx = int(len(df) * 0.8)
train_df = df.iloc[:split_idx]
test_df = df.iloc[split_idx:]

print(f"📊 Train: {len(train_df):,} rows ({train_df['date'].min()} to {train_df['date'].max()})")
print(f"📊 Test:  {len(test_df):,} rows ({test_df['date'].min()} to {test_df['date'].max()})")

📊 Train: 82,121 rows (2024-06-15 00:00:00 to 2025-09-05 00:00:00)
📊 Test:  20,531 rows (2025-09-05 00:00:00 to 2025-12-24 00:00:00)


In [7]:
# Extract features and target
X_train = train_df[FEATURE_COLS].values
y_train = train_df[TARGET].values
X_test = test_df[FEATURE_COLS].values
y_test = test_df[TARGET].values

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"✅ Features scaled")

✅ Features scaled


In [8]:
# Convert to PyTorch tensors
X_train_t = torch.FloatTensor(X_train_scaled).to(device)
y_train_t = torch.FloatTensor(y_train).unsqueeze(1).to(device)
X_test_t = torch.FloatTensor(X_test_scaled).to(device)
y_test_t = torch.FloatTensor(y_test).unsqueeze(1).to(device)

# DataLoaders
train_ds = TensorDataset(X_train_t, y_train_t)
train_loader = DataLoader(train_ds, batch_size=256, shuffle=True)

print(f"✅ Tensors on {device}")

✅ Tensors on cuda


## 4. Define V2 Model

In [9]:
class LocationMLP(nn.Module):
    """Location-based MLP for weather prediction."""
    def __init__(self, input_dim, hidden_dims=[256, 128, 64], dropout=0.3):
        super().__init__()
        layers = []
        prev_dim = input_dim
        for h in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, h),
                nn.BatchNorm1d(h),
                nn.ReLU(),
                nn.Dropout(dropout)
            ])
            prev_dim = h
        layers.append(nn.Linear(prev_dim, 1))
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)

# Initialize model
model = LocationMLP(
    input_dim=len(FEATURE_COLS),
    hidden_dims=[256, 128, 64],
    dropout=0.3
).to(device)

print(f"📊 Model parameters: {sum(p.numel() for p in model.parameters()):,}")
print(model)

📊 Model parameters: 49,281
LocationMLP(
  (network): Sequential(
    (0): Linear(in_features=27, out_features=256, bias=True)
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=256, out_features=128, bias=True)
    (5): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.3, inplace=False)
    (8): Linear(in_features=128, out_features=64, bias=True)
    (9): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): Dropout(p=0.3, inplace=False)
    (12): Linear(in_features=64, out_features=1, bias=True)
  )
)


## 5. Training

In [10]:
# Training setup
criterion = nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.5)

# Training loop
epochs = 100
best_val_loss = float('inf')
patience = 15
patience_counter = 0
history = {'train_loss': [], 'val_loss': []}

print("🚀 Starting training...")
for epoch in range(epochs):
    # Train
    model.train()
    train_losses = []
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        pred = model(X_batch)
        loss = criterion(pred, y_batch)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())
    
    # Validate
    model.eval()
    with torch.no_grad():
        val_pred = model(X_test_t)
        val_loss = criterion(val_pred, y_test_t).item()
    
    train_loss = np.mean(train_losses)
    history['train_loss'].append(train_loss)
    history['val_loss'].append(val_loss)
    
    scheduler.step(val_loss)
    
    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        best_state = model.state_dict().copy()
    else:
        patience_counter += 1
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1:3d} | Train: {train_loss:.4f} | Val: {val_loss:.4f} | LR: {optimizer.param_groups[0]['lr']:.6f}")
    
    if patience_counter >= patience:
        print(f"\n⏹️ Early stopping at epoch {epoch+1}")
        break

# Restore best model
model.load_state_dict(best_state)
print(f"\n✅ Best validation loss: {best_val_loss:.4f}")

🚀 Starting training...
Epoch  10 | Train: 12.3184 | Val: 5.5532 | LR: 0.001000
Epoch  20 | Train: 11.6508 | Val: 5.5041 | LR: 0.000500
Epoch  30 | Train: 11.0884 | Val: 5.6170 | LR: 0.000250

⏹️ Early stopping at epoch 36

✅ Best validation loss: 5.5035


In [11]:
# Plot training history
fig = go.Figure()
fig.add_trace(go.Scatter(y=history['train_loss'], name='Train'))
fig.add_trace(go.Scatter(y=history['val_loss'], name='Validation'))
fig.update_layout(title='📉 Training History', xaxis_title='Epoch', yaxis_title='MSE Loss',
                  paper_bgcolor='#0f0f1a', plot_bgcolor='#0f0f1a', font_color='white')
fig.show()

## 6. Evaluation

In [12]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

model.eval()
with torch.no_grad():
    y_pred = model(X_test_t).cpu().numpy().flatten()

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("="*50)
print("📊 V2 Location Model - Test Results")
print("="*50)
print(f"MAE:  {mae:.2f}°C")
print(f"RMSE: {rmse:.2f}°C")
print(f"R²:   {r2:.4f}")

📊 V2 Location Model - Test Results
MAE:  1.60°C
RMSE: 2.35°C
R²:   0.9307


In [13]:
# Actual vs Predicted scatter
fig = px.scatter(x=y_test, y=y_pred, opacity=0.3,
                 labels={'x': 'Actual (°C)', 'y': 'Predicted (°C)'},
                 title=f'🎯 V2 Model: Actual vs Predicted (MAE={mae:.2f}°C)')
fig.add_trace(go.Scatter(x=[-20, 50], y=[-20, 50], mode='lines', name='Perfect',
                         line=dict(color='red', dash='dash')))
fig.update_layout(paper_bgcolor='#0f0f1a', plot_bgcolor='#0f0f1a', font_color='white')
fig.show()

In [14]:
# Error by climate zone
test_df_eval = test_df.copy()
test_df_eval['prediction'] = y_pred
test_df_eval['error'] = abs(test_df_eval['temperature_celsius'] - test_df_eval['prediction'])

# Get climate zone names
zone_names = {0: 'Tropical', 1: 'Subtropical', 2: 'Temperate', 3: 'Continental', 4: 'Polar'}
test_df_eval['climate_zone'] = test_df_eval['climate_zone_encoded'].map(zone_names)

zone_mae = test_df_eval.groupby('climate_zone')['error'].mean().round(2)
print("\n📊 MAE by Climate Zone:")
print(zone_mae)


📊 MAE by Climate Zone:
climate_zone
Continental    2.24
Subtropical    1.68
Temperate      2.25
Tropical       1.12
Name: error, dtype: float64


## 7. Save Model Artifacts

In [15]:
# Save model
checkpoint = {
    'model_state_dict': model.state_dict(),
    'input_dim': len(FEATURE_COLS),
    'hidden_dims': [256, 128, 64],
    'dropout': 0.3,
    'feature_cols': FEATURE_COLS,
    'metrics': {'mae': mae, 'rmse': rmse, 'r2': r2}
}
torch.save(checkpoint, '../models/location_model.pt')
print("✅ Saved location_model.pt")

# Save scaler
joblib.dump(scaler, '../models/location_scaler.joblib')
print("✅ Saved location_scaler.joblib")

# Save model config
config = {
    'version': '2.0',
    'feature_cols': FEATURE_COLS,
    'input_dim': len(FEATURE_COLS),
    'hidden_dims': [256, 128, 64],
    'dropout': 0.3,
    'metrics': {'mae': round(mae, 2), 'rmse': round(rmse, 2), 'r2': round(r2, 4)}
}
with open('../models/model_config.json', 'w') as f:
    json.dump(config, f, indent=2)
print("✅ Saved model_config.json")

✅ Saved location_model.pt
✅ Saved location_scaler.joblib
✅ Saved model_config.json


## 8. Summary

In [16]:
print("="*60)
print("🎉 V2 Location Model Training Complete!")
print("="*60)
print(f"\n📊 Model Performance:")
print(f"   • MAE:  {mae:.2f}°C")
print(f"   • RMSE: {rmse:.2f}°C")
print(f"   • R²:   {r2:.4f}")
print(f"\n📦 Artifacts Saved:")
print(f"   • v2/models/location_model.pt")
print(f"   • v2/models/location_scaler.joblib")
print(f"   • v2/models/model_config.json")
print(f"\n🚀 Ready to integrate with web app!")

🎉 V2 Location Model Training Complete!

📊 Model Performance:
   • MAE:  1.60°C
   • RMSE: 2.35°C
   • R²:   0.9307

📦 Artifacts Saved:
   • v2/models/location_model.pt
   • v2/models/location_scaler.joblib
   • v2/models/model_config.json

🚀 Ready to integrate with web app!
